In [1]:
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests as req
from tqdm import tqdm
import os
import re
from tika import parser
import shutil
from os import listdir
from os.path import isfile, join

In [2]:
# 제목
title_list = []
# 날짜
date_list = []
# 첨부 파일
f_name_list = [] # 파일명
link_list = [] # url
# text 변환
txt_list = []

In [3]:
# HTML 컨텐츠 수집
for i in tqdm(range(1, 54)) : 
    min_url = f"https://www.bok.or.kr/portal/bbs/B0000245/list.do?menuNo=200761&pageIndex={i}"
    res = req.get(min_url)
    soup = bs(res.content, "html.parser")
    
    links = soup.select("div.col.m2.s2.x3.fileLink a")
    titles = soup.select("span.titlesub")
    dates = soup.select("span.date")

    for i in range(len(titles)) :
        title_list.append(titles[i].text)
    for i in range(len(dates)) :
        date_list.append(dates[i].text)

    cnt = 0 # 중요! : i번 째 페이지에서 컨텐츠 갯수만큼 돌고, 다시 초기화하여 다음 페이지로 넘어가기 위함
    for i in links :
        filename = i.get_text().strip()
        
        # 확장자가 hwp인 파일만 수집
        if filename[-3:] == "pdf" :
            # 파일명
            filename = f"minutes_{dates[cnt].text}.pdf"
            f_name_list.append(filename)
            cnt += 1
            
            # 파일의 주소 값
            file_addr = "http://bok.or.kr" + i["href"]
            link_list.append(file_addr)

print(len(title_list), len(date_list), len(f_name_list), len(link_list))

100%|██████████| 53/53 [01:12<00:00,  1.37s/it]

526 526 393 393


In [4]:
# 한국은행 금융통화위원회 의사록 페이지 HTML구조가 바뀌기 전까지로 list 길이 수정
title_list = title_list[:300]
date_list = date_list[:300]
f_name_list = f_name_list[:300]
link_list = link_list[:300]

## PDF

In [5]:
# 첨부파일 다운로드
for i in tqdm(range(len(link_list))) :
    try :
        res = req.get(link_list[i])
        # dir 생성 or 확인
        if not os.path.isdir("data") :
            os.mkdir("data")
        if res.status_code == 200 :
            file_path = os.path.join("data", f_name_list[i])

            # 파일 열기, 쓰기
            open(file_path, "wb").write(res.content)
    except Exception as e :
        print("Error! : ", e)
        continue
print("Complete😇")

100%|██████████| 300/300 [00:54<00:00,  5.49it/s]

Complete😇


## PDF 👉 text

In [6]:
# 폴더 내 pdf파일을 txt 파일로 변경
def pdf2txt(source_folder="./data/", output_folder="./data/text/"):
    if not os.path.isdir("./data/text/") :
            os.mkdir("./data/text/")
    # 지정 폴더 내 파일 목록 조회 (파일만)
    pdf_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    
    try :
        for pdf in tqdm(pdf_files) :
            pdf_filepath = source_folder + pdf
            pdf_tmp_filepath = output_folder + 'tmp.pdf'

            # pdf 파일을 text로 변환
            shutil.copyfile(pdf_filepath, pdf_tmp_filepath)
            parsedPDF = parser.from_file(pdf_tmp_filepath)["content"]

            #enter 삭제
            parsedPDF = re.sub('\n', '', parser.from_file(pdf_tmp_filepath)["content"])

            output_filepath = (output_folder + pdf).replace('.pdf', '.txt')
            
            with open(output_filepath, "w", -1,"utf-8") as f:
                print(output_filepath)
                f.write(parsedPDF)
                f.close()
    except Exception as e:
        print('오류', e)
        pass

In [7]:
pdf2txt(source_folder="./data/", output_folder="./data/text/")

  0%|          | 1/297 [00:08<41:10,  8.35s/it]

./data/text/minutes_등록일2009.06.23.txt


  1%|          | 2/297 [00:08<17:32,  3.57s/it]

./data/text/minutes_등록일2009.07.07.txt


  1%|          | 3/297 [00:09<10:58,  2.24s/it]

...

./data/text/minutes_등록일2024.04.16.txt


100%|█████████▉| 296/297 [01:45<00:00,  2.74it/s]

./data/text/minutes_등록일2024.04.30.txt
./data/text/minutes_등록일2024.05.28.txt


100%|██████████| 297/297 [01:45<00:00,  2.80it/s]

./data/text/minutes_등록일2024.06.11.txt


### 의사록 섹션 분리

In [8]:
def tidy_sentences(section) :
    sentence_enders = re.compile(r'((?<=[함음됨임봄짐움])(\s*\n|\.|;)|(?<=다)\.)\s*')
    splits = list((m.start(), m.end()) for m in re.finditer(sentence_enders, section))
    starts = [0] + [i[1] for i in splits]
    ends = [i[0] for i in splits]
    sentences = [section[start:end] for start, end in zip(starts[:-1], ends)]
    for i, s in enumerate(sentences):
        sentences[i] = (s.replace('\n', ' ').replace(' ', ' ')) + '.'
    text = '\n'.join(sentences) if len(sentences) > 0 else ''
    
    return sentences, text

In [9]:
def preprocess_minutes(minutes) :
    pos = re.search('(.?국내외\s?경제\s?동향.?과 관련하여,?|\(가\).+경제전망.*|\(가\) 국내외 경제동향 및 평가)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s1 = pos.start() if pos else -1
    pos = re.search('(.?외환.?국제금융\s?동향.?과 관련하여.*|\(나\) 외환.국제금융\s?(및 금융시장)?\s?동향)\n?\s*(일부 위원은|대부분의 위원들은)', minutes,re.MULTILINE)
    s2 = pos.start() if pos else -1
    pos = re.search('(.?금융시장\s?동향.?과 관련하여,?|\(다\) 금융시장\s?동향)\n?\s*일부 위원은', minutes, re.MULTILINE)
    s3 = pos.start() if pos else -1
    pos = re.search('((\((다|라)\) )?.?통화정책\s?방향.?에 관한 토론,?|이상과 같은 의견\s?교환을 바탕으로.*통화정책\s?방향.*에.*토론.*)\n?', minutes,re.MULTILINE)
    s4 = pos.start() if pos else -1
    pos = re.search('(\(4\) 정부측 열석자 발언.*)\n?', minutes, re.MULTILINE)
    s5 = pos.start() if pos else -1
    pos = re.search('(\(.*\) 한국은행 기준금리 결정에 관한 위원별 의견\s?개진|이상과 같은 토론에 이어 .* 관한 위원별 의견개진이 있었음.*)\n?', minutes,re.MULTILINE)
    s6 = pos.start() if pos else -1
    positer = re.finditer('(\(\s?.*\s?\) ()(심의결과|토의결론))\n?', minutes, re.MULTILINE)
    s7 = [pos.start() for pos in positer if pos.start() > s6]
    s7 = s7[0] if s7 else -1

    # 국내외 경제동향
    bos = s1
    eos = s2
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section1, section1_txt = tidy_sentences(section)

    # 외환․국제금융 동향
    bos = s2
    eos = s3 if s3 >= 0 else s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section2, section2_txt = tidy_sentences(section)

    # 금융시장 동향
    bos = s3
    eos = s4
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section3, section3_txt = tidy_sentences(section)

    # 통화정책방향
    bos = s4
    eos = s5 if s5 >= 0 else s6 if s6 >= 0 else s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section4, section4_txt = tidy_sentences(section)

    # 위원별 의견 개진
    bos = s6
    eos = s7
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('(일부|대부분의) 위원들?은', section, re.MULTILINE)
    bos = pos.start() if pos else -1
    section = section[bos:] if bos >= 0 else section
    section5, section5_txt = tidy_sentences(section)

    # 정부측 열석자 발언
    bos = s5
    eos = s6
    section = minutes[bos:eos] if bos >= 0 or eos >= 0 else ''
    pos = re.search('정부측 열석자 발언', section, re.MULTILINE)
    bos = pos.end() + 1 if pos else -1
    section = section[bos:] if bos >= 0 else section
    section6, section6_txt = tidy_sentences(section)

    sections = ['Economic Situation', 'Foreign Currency', 'Financial Markets',
                'Monetary Policy', 'Participants’ Views', 'Government’s View']
    section_texts = (section1, section2, section3, section4, section5, section6)

    return sections, section_texts

In [10]:
def preprocessing(source_folder="/data/text/", output_file="/data/min_df.csv") :
    # 지정 폴더 내 파일 목록 조회 (파일만)
    txt_files = [f for f in listdir(source_folder) if isfile(join(source_folder, f))]
    txt_files.sort()
    df = pd.DataFrame(columns=["date", "minutes"])
    df["Economic Situation"] = ""
    df["Foreign Currency"] = ""
    df["Financial Markets"] = ""
    df["Monetary Policy"] = ""
    df["Participant Views"] = ""
    df["Government View"] = ""

    df["Economic Situation count"] = ""
    df["Foreign Currency count"] = ""
    df["Financial Markets count"] = ""
    df["Monetary Policy count"] = ""
    df["Participant Views count"] = ""
    df["Government View count"] = ""

    for txt_file in txt_files :
        try :
            with open(source_folder + txt_file, "r", encoding = "utf-8") as f :
                txt = f.read()
                sections, section_texts = preprocess_minutes(txt)

                print(txt_file)

                df.loc[len(df)] = [txt_file.split('_')[0],
                                   txt,
                                   '@@@'.join(section_texts[0]),
                                   '@@@'.join(section_texts[1]),
                                   '@@@'.join(section_texts[2]),
                                   '@@@'.join(section_texts[3]),
                                   '@@@'.join(section_texts[4]),
                                   '@@@'.join(section_texts[5]),
                                   len(section_texts[0]),
                                   len(section_texts[1]),
                                   len(section_texts[2]),
                                   len(section_texts[3]),
                                   len(section_texts[4]),
                                   len(section_texts[5]),]
        except Exception as e:
            print('오류', e)
            pass

    df.to_csv(output_file, index = False, encoding = "utf-8", errors = 'ignore')
    print("DF 구조 ", df.shape)
    
    return df

result = preprocessing(source_folder="./data/text/", output_file="./data/min_df.csv")

minutes_등록일2009.06.23.txt
minutes_등록일2009.07.07.txt
minutes_등록일2009.07.28.txt
minutes_등록일2009.08.11.txt
minutes_등록일2009.08.25.txt
minutes_등록일2009.09.08.txt
minutes_등록일2009.09.29.txt
minutes_등록일2009.10.27.txt
minutes_등록일2009.11.10.txt
minutes_등록일2009.11.24.txt
minutes_등록일2009.12.29.txt
minutes_등록일2010.01.12.txt
minutes_등록일2010.01.26.txt
minutes_등록일2010.02.09.txt
minutes_등록일2010.02.23.txt
minutes_등록일2010.03.09.txt
minutes_등록일2010.03.30.txt
minutes_등록일2010.04.27.txt
minutes_등록일2010.05.11.txt
minutes_등록일2010.05.25.txt
minutes_등록일2010.06.08.txt
minutes_등록일2010.06.29.txt
minutes_등록일2010.07.27.txt
minutes_등록일2010.08.10.txt
minutes_등록일2010.08.24.txt
minutes_등록일2010.09.14.txt
minutes_등록일2010.09.28.txt
minutes_등록일2010.10.19.txt
minutes_등록일2010.10.26.txt
minutes_등록일2010.11.09.txt
minutes_등록일2010.11.30.txt
minutes_등록일2011.01.04.txt
minutes_등록일2011.01.25.txt
minutes_등록일2011.02.08.txt
minutes_등록일2011.02.22.txt
minutes_등록일2011.03.02.txt
minutes_등록일2011.03.08.txt
minutes_등록일2011.03.29.txt
minutes_등록일2

In [11]:
result.to_csv("./data/min_df.csv", index = False)

In [12]:
pd.read_csv("./data/min_df.csv")

,date,minutes,Economic Situation,Foreign Currency,Financial Markets,Monetary Policy,Participant Views,Government View,Economic Situation count,Foreign Currency count,Financial Markets count,Monetary Policy count,Participant Views count,Government View count
0,minutes,UYWVWGTCRIIL.hwp금융통화위원회 의사록2009년도 제 차 회의12일 자1...,NaN,NaN,NaN,NaN,일부 위원은 이번 달에는 한국은행 기준금리를 현 수준으로 유지할 것을제안하면서 다음...,NaN,0,0,0,0,4,0
1,minutes,WMLHEQIJNQNW.hwp- 1 -금융통화위원회 의사록2009년도 제 차 회의1...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
2,minutes,AXTJVNBXVDZE.hwp- 1 -금융통화위원회 의사록2009년도 제 차 회의1...,NaN,NaN,NaN,NaN,일부 위원은 다음과 같은 점을 종합적으로 고려할 때 당분간 통화정책은경기회복을 적극...,NaN,0,0,0,0,4,0
3,minutes,HVBEHWQPSRCR.hwp- 1 -금융통화위원회 의사록2009년도 제 차 회의1...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
4,minutes,PTKUFAIJRVLU.hwp금융통화위원회 의사록2009년도 제 차 회의16일 자1...,NaN,NaN,NaN,NaN,일부 위원은 다음과 같은 점을 종합적으로 고려할 때 앞으로 당분간 통화정책 운용은 ...,NaN,0,0,0,0,3,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292,minutes,2024년도 제5차금융통화위원회(정기) 의사록한 국 은 행- 1 -1. 일 자 20...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
293,minutes,2024년도 제6차금융통화위원회(정기) 의사록한 국 은 행- 1 -1. 일 자 20...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0
294,minutes,2024년도 제7차금융통화위원회(정기) 의사록한 국 은 행- 1 -1. 일 자 20...,일부 위원은 소비자물가 상승률에 대한 식료품의 기여도가 미국이나 유로지역에서는 지속...,일부 위원은 미국 소비자물가(2024.3월) 발표 이후 일각에서 미 연준의 금리인상...,NaN,NaN,일부 위원은 소비자물가 상승률에 대한 식료품의 기여도가 미국이나 유로지역에서는 지속...,NaN,49,57,0,0,307,0
295,minutes,2024년도 제9차금융통화위원회(정기) 의사록한 국 은 행- 1 -1. 일 자 20...,NaN,NaN,NaN,NaN,NaN,NaN,0,0,0,0,0,0


In [13]:
# DataFrame 생성
pd.set_option("display.max_rows", None)
df = pd.DataFrame({
    "title" : title_list,
    "date" : date_list,
    "file" : f_name_list,
    "url" : link_list
})

# 날짜 컬럼 문자열 자르기
df["date"] = df["date"].str.replace("등록일", "")
df

,title,date,file,url
0,금융통화위원회 의사록(2024년도 제10차)(2024.5.23),2024.06.11,minutes_등록일2024.06.11.pdf,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
1,금융통화위원회 의사록(2024년도 제9차)(2024.5.9),2024.05.28,minutes_등록일2024.05.28.pdf,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
2,금융통화위원회 의사록(2024년도 제7차)(2024.4.12),2024.04.30,minutes_등록일2024.04.30.pdf,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
3,금융통화위원회 의사록(2024년도 제6차)(2024.3.28),2024.04.16,minutes_등록일2024.04.16.pdf,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
4,금융통화위원회 의사록(2024년도 제5차)(2024.3.14),2024.04.02,minutes_등록일2024.04.02.pdf,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
5,금융통화위원회 의사록(2024년도 제4차)(2024.2.22),2024.03.12,minutes_등록일2024.03.12.pdf,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
6,금융통화위원회 의사록(2024년도 제3차)(2024.2.8),2024.02.27,minutes_등록일2024.02.27.pdf,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
7,금융통화위원회 의사록(2024년도 제2차)(2024.1.25),2024.02.13,minutes_등록일2024.02.13.pdf,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
8,금융통화위원회 의사록(2024년도 제1차)(2024.1.11),2024.01.30,minutes_등록일2024.01.30.pdf,http://bok.or.kr/portal/cmmn/file/fileDown.do?...
9,금융통화위원회 의사록(2023년도 제24차)(2023.12.28),2024.01.16,minutes_등록일2024.01.16.pdf,http://bok.or.kr/portal/cmmn/file/fileDown.do?...


In [14]:
df.to_csv("data/minutes_info.csv", encoding = "utf8")